<a href="https://colab.research.google.com/github/alenready/ML_AI_ICT-Assignments/blob/main/ThetaBot_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:
import gradio as gr
import tensorflow as tf
import numpy as np
import requests
import base64
from io import BytesIO
from PIL import Image

# Import the models and preprocessing functions for each architecture:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input as resnet_preprocess, decode_predictions
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input as inception_preprocess
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input as efficientnet_preprocess


2. Load (or Fall Back) to Models
In an ideal scenario you’d have models fine-tuned on Food-101 saved as “resnet_food101.h5”, “inception_food101.h5”, and “efficientnet_food101.h5”. This example tries to load those first. If they aren’t available, it will fall back to the standard ImageNet‑trained models.

In [3]:
def load_model_or_fallback(model_name, food101_path, default_fn):
    try:
        # Attempt to load a Food-101 fine-tuned model (if you have one)
        model = tf.keras.models.load_model(food101_path)
        print(f"{model_name} loaded from Food-101 fine-tuned weights!")
    except Exception as e:
        # Fallback: load the standard model pre-trained on ImageNet.
        print(f"{model_name} Food-101 weights not found, falling back to ImageNet weights.")
        model = default_fn(weights="imagenet")
    return model

model_resnet = load_model_or_fallback("ResNet50", "resnet_food101.h5", ResNet50)
model_inception = load_model_or_fallback("InceptionV3", "inception_food101.h5", InceptionV3)
model_efficientnet = load_model_or_fallback("EfficientNetB0", "efficientnet_food101.h5", EfficientNetB0)


ResNet50 Food-101 weights not found, falling back to ImageNet weights.
102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
InceptionV3 Food-101 weights not found, falling back to ImageNet weights.
96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
EfficientNetB0 Food-101 weights not found, falling back to ImageNet weights.
21834768/21834768 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


3. Decide on the Label Space
We want to know whether our models output predictions for Food-101 (101 classes) or for ImageNet (1000 classes). If they’re Food-101 models, we’ll use a custom list of food class names; otherwise, we use the built‑in decode_predictions.

In [4]:
# Check output shape of one model (assumed same for all three)
if model_resnet.output_shape[-1] == 101:
    use_food101 = True
    # List of Food-101 labels (101 classes)
    food101_labels = [
        "apple_pie", "baby_back_ribs", "baklava", "beef_carpaccio", "beef_tartare",
        "beet_salad", "beignets", "bibimbap", "bread_pudding", "breakfast_burrito",
        "bruschetta", "caesar_salad", "cannoli", "caprese_salad", "carrot_cake",
        "ceviche", "cheesecake", "chicken_curry", "chicken_quesadilla", "chicken_wings",
        "chocolate_cake", "chocolate_mousse", "churros", "clam_chowder", "club_sandwich",
        "crab_cakes", "creme_brulee", "croque_madame", "cup_cakes", "deviled_eggs",
        "donuts", "dumplings", "edamame", "eggs_benedict", "escargots", "falafel",
        "filet_mignon", "fish_and_chips", "foie_gras", "french_fries", "french_onion_soup",
        "fried_calamari", "fried_rice", "frozen_yogurt", "garlic_bread", "gnocchi",
        "greek_salad", "grilled_cheese_sandwich", "grilled_salmon", "guacamole",
        "hamburger", "hot_and_sour_soup", "hot_dog", "huevos_rancheros", "ice_cream",
        "lasagna", "lobster_bisque", "lobster_roll_sandwich", "macaroni_and_cheese",
        "macarons", "miso_soup", "mussels", "nachos", "omelette", "onion_rings",
        "oysters", "pad_thai", "paella", "pancakes", "panna_cotta", "peking_duck",
        "pho", "pizza", "pork_chop", "poutine", "prime_rib", "pulled_pork_sandwich",
        "ramen", "ravioli", "red_velvet_cake", "risotto", "samosa", "sashimi",
        "schnitzel", "scallops", "seaweed_salad", "shrimp_and_grits", "spaghetti_bolognese",
        "spaghetti_carbonara", "spring_rolls", "steak", "strawberry_shortcake", "sushi",
        "tacos", "takoyaki", "tiramisu", "tuna_tartare", "waffles"
    ]
else:
    use_food101 = False


4. Define Helper Function to Preprocess Images
Different models expect different image sizes and preprocessing. The function below converts an image (from the user) into the right format for each model.

In [5]:
def preprocess_image_for_model(image, target_size, preprocess_func):
    # If image is not already a PIL image, convert it.
    if not isinstance(image, Image.Image):
        image = Image.fromarray(image.astype("uint8"), "RGB")
    # Resize the image to the target size (e.g., 224x224 or 299x299)
    image = image.resize(target_size)
    # Convert the image to a NumPy array and add a batch dimension.
    x = np.array(image)
    x = np.expand_dims(x, axis=0)
    # Preprocess (scale/normalize) the image as required by the model.
    x = preprocess_func(x)
    return x


5. Ensemble Classifier Function for Food Images
This function uses all three models to predict what food is in an image. It preprocesses the image for each model (using its expected size and preprocessing method), gets each model’s prediction, and then averages them. Finally, it decodes the averaged prediction into a human‑readable result.

python
Copy
Edit


In [6]:
def ensemble_classify_food_image(image):
    """
    Processes an image through three models (ResNet50, InceptionV3, EfficientNetB0),
    averages their predictions, and returns the top 3 predicted food classes.
    """
    # Preprocess the image for each model:
    x_resnet = preprocess_image_for_model(image, (224, 224), resnet_preprocess)
    x_inception = preprocess_image_for_model(image, (299, 299), inception_preprocess)
    x_efficient = preprocess_image_for_model(image, (224, 224), efficientnet_preprocess)

    # Get predictions from each model:
    pred_resnet = model_resnet.predict(x_resnet)
    pred_inception = model_inception.predict(x_inception)
    pred_efficient = model_efficientnet.predict(x_efficient)

    # Average the predictions from all three models:
    ensemble_pred = (pred_resnet + pred_inception + pred_efficient) / 3.0

    # Decode the predictions:
    if use_food101:
        # For Food-101, use our custom label list.
        top_indices = np.argsort(ensemble_pred[0])[::-1][:3]
        result = ""
        for idx in top_indices:
            label = food101_labels[idx]
            prob = ensemble_pred[0][idx] * 100
            result += f"{label}: {prob:.2f}%\n"
    else:
        # For ImageNet models, use decode_predictions.
        decoded = decode_predictions(ensemble_pred, top=3)[0]
        result = "\n".join([f"{desc}: {prob*100:.2f}%" for (_, desc, prob) in decoded])
    return result


6. Helper Function to Convert a PIL Image to a Base64 String
This is used to display images in the chatbot via Markdown.

In [7]:
def pil_to_base64(img):
    buffered = BytesIO()
    img.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return f"data:image/png;base64,{img_str}"


7. Functions to Generate Chat Responses
a. For Text Queries
This function provides canned responses (or sample food images) based on the user’s text message.

In [8]:
def get_text_response(user_message):
    """
    For text queries, if the message includes phrases like "show me pizza",
    then a sample image is fetched. Otherwise, a generic food-related response is returned.
    """
    if "show me" in user_message.lower():
        parts = user_message.lower().split("show me")
        if len(parts) > 1:
            food_item = parts[1].strip().split()[0]
            # Dictionary of sample images for some food items.
            sample_images = {
                "pizza": "https://upload.wikimedia.org/wikipedia/commons/d/d3/Supreme_pizza.jpg",
                "burger": "https://upload.wikimedia.org/wikipedia/commons/4/4f/Hamburger_%28black_bg%29.jpg",
                "salad": "https://upload.wikimedia.org/wikipedia/commons/6/66/Salad_platter.jpg",
                "sushi": "https://upload.wikimedia.org/wikipedia/commons/6/60/Sushi_platter.jpg"
            }
            if food_item in sample_images:
                response_text = f"Here is an image of {food_item}:"
                try:
                    response_image = Image.open(requests.get(sample_images[food_item], stream=True).raw)
                    data_url = pil_to_base64(response_image)
                    image_md = f"![{food_item}]({data_url})"
                    full_response = response_text + "\n" + image_md
                    return full_response, None
                except Exception as e:
                    return f"Sorry, there was an error loading the image for {food_item}.", None
            else:
                return f"Sorry, I don't have a sample image for '{food_item}'.", None

    if "calorie" in user_message.lower():
        return "The calorie content depends on ingredients. Could you specify the dish?", None
    elif "recipe" in user_message.lower():
        return "I can help with recipes! What dish are you interested in?", None
    else:
        return ("I'm here to help with food-related queries! You can ask me about food, recipes, "
                "or upload a food image for classification."), None


b. Chatbot Callback Functions
These functions update the conversation history when the user sends text or uploads an image.

In [9]:
def chat_response(user_message, history):
    """
    Triggered when the user sends a text message.
    It gets a response and updates the conversation history.
    """
    bot_response, _ = get_text_response(user_message)
    history = history + [[user_message, bot_response]]
    # Return an empty text box (to clear input) and updated history.
    return "", history, history

def classify_uploaded_image(image, history):
    """
    Triggered when the user uploads an image.
    It classifies the image using the ensemble of models and updates the conversation history.
    """
    if image is None:
        return history, history

    # Get predictions from the ensemble classifier.
    classification_result = ensemble_classify_food_image(image)

    # Convert the uploaded image to a format that can be displayed in the chat.
    if isinstance(image, np.ndarray):
        pil_img = Image.fromarray(image.astype("uint8"), "RGB")
    else:
        pil_img = image
    data_url = pil_to_base64(pil_img)
    image_md = f"![Uploaded Image]({data_url})"

    bot_message = (f"{image_md}\nI have analyzed the image. Here are the predictions:\n"
                   f"{classification_result}")
    # For the conversation, we show the uploaded image as the user message.
    user_message = image_md
    history = history + [[user_message, bot_message]]
    return history, history


8. Build and Launch the Gradio Interface
This final cell sets up the interactive chatbot. You can enter text or upload an image, and the respective functions are triggered.

In [10]:
with gr.Blocks() as demo:
    gr.Markdown("## Food Chatbot & Ensemble Image Classifier")
    gr.Markdown(
        "Ask me food-related questions (e.g., 'What is the calorie content of an avocado?') or type 'show me pizza' for a sample image. "
        "You can also upload a food image for classification."
    )

    # Shared state for the conversation history.
    state = gr.State([])

    # Chat display area (shows the conversation as a list of message pairs).
    chatbot = gr.Chatbot()

    with gr.Row():
        txt = gr.Textbox(placeholder="Enter your message here", label="Chat Input")
        send_button = gr.Button("Send")

    # Image upload component.
    img_input = gr.Image(type="numpy", label="Upload Food Image for Classification")

    # When "Send" is clicked, update the chat with the text response.
    send_button.click(fn=chat_response,
                      inputs=[txt, state],
                      outputs=[txt, chatbot, state])

    # When an image is uploaded, classify it and update the chat.
    img_input.change(fn=classify_uploaded_image,
                     inputs=[img_input, state],
                     outputs=[chatbot, state])

demo.launch()


/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4729bca723885eba59.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


An ensemble food image classifier using three models—InceptionV3, EfficientNetB0, and ResNet50—each ideally fine‐tuned on the Food-101 dataset. (If you don’t have Food‑101–fine‐tuned weights, the code will fall back to ImageNet‑trained weights.) This ensemble approach combines the predictions from all three models to give you a more accurate (and in some cases faster) prediction. We’ve also optimized the code for a quicker response.